In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import TFTModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import rmse, mae, r2_score

os.environ.pop("PYTORCH_ENABLE_MPS_FALLBACK", None)

# ============================
# 1. 데이터 불러오기
df = pd.read_csv("total_data.csv")
df["Date"] = pd.to_datetime(df["Date"], format="%Y.%m.%d")
df = df.sort_values("Date").reset_index(drop=True)

# ============================
# 2. Business day 기준 리샘플링 
df = df.set_index("Date").asfreq("B")               
df = df.reset_index()

# ============================
# 3. 모든 수치형 칼럼을 float32로 변환 (MPS 호환성)
for col in df.select_dtypes(include=["float64"]).columns:
    df[col] = df[col].astype("float32")

df

,Date,콜금리(1일),KORIBOR(3개월),KORIBOR(6개월),KORIBOR(12개월),CD(91일),국고채(3년),국고채(10년),KOFR,기준금리,...,실업률,경제활동참가율,EFFR,제조업 PMI,기대 인플레이션율,현재경기판단CSI,향후1년경기전망CSI,업황실적BSI(전산업),업황실적BSI(제조업),Tone_score
0,2018-01-02,1.520,1.70,1.84,1.99,1.66,2.119,2.489,1.649,1.5,...,3.3,63.200001,1.42,50.700001,2.6,90.0,102.0,78.0,77.0,0.165511
1,2018-01-03,1.500,1.68,1.81,1.96,1.66,2.127,2.508,1.611,1.5,...,3.3,63.200001,1.42,50.700001,2.6,90.0,102.0,78.0,77.0,0.238423
2,2018-01-04,1.480,1.67,1.80,1.95,1.66,2.142,2.523,1.565,1.5,...,3.3,63.200001,1.42,50.700001,2.6,90.0,102.0,78.0,77.0,0.102867
3,2018-01-05,1.470,1.66,1.79,1.95,1.66,2.099,2.518,1.559,1.5,...,3.3,63.200001,1.42,50.700001,2.6,90.0,102.0,78.0,77.0,0.150966
4,2018-01-08,1.460,1.65,1.77,1.94,1.66,2.135,2.540,1.498,1.5,...,3.3,63.200001,1.42,50.700001,2.6,90.0,102.0,78.0,77.0,0.212867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973,2025-07-25,2.515,2.50,2.52,2.53,2.51,2.485,2.855,2.496,3.5,...,2.5,64.400002,4.33,48.000000,2.5,86.0,106.0,68.0,68.0,0.176624
1974,2025-07-28,2.506,2.51,2.52,2.53,2.51,2.464,2.829,2.493,3.5,...,2.5,64.400002,4.33,48.000000,2.5,86.0,106.0,68.0,68.0,0.184720
1975,2025-07-29,2.500,2.51,2.52,2.54,2.51,2.460,2.823,2.493,3.5,...,2.5,64.400002,4.33,48.000000,2.5,86.0,106.0,68.0,68.0,0.101873
1976,2025-07-30,2.518,2.51,2.52,2.53,2.51,2.454,2.797,2.496,3.5,...,2.5,64.400002,4.33,48.000000,2.5,86.0,106.0,68.0,68.0,0.017564


In [10]:
# 4. TimeSeries 객체 변환
series = TimeSeries.from_dataframe(df, time_col="Date", value_cols="KOFR")

covariates = TimeSeries.from_dataframe(
    df,
    time_col="Date",
    value_cols=[
        '콜금리(1일)', 'KORIBOR(3개월)', 'KORIBOR(6개월)', 'KORIBOR(12개월)',
        'CD(91일)', '국고채(3년)', '국고채(10년)', '기준금리',
        'CPI', 'Core CPI', '전산업생산지수', '원화실질실효환율',
        '실업률', '경제활동참가율', 'EFFR', '제조업 PMI',
        '기대 인플레이션율', '현재경기판단CSI', '향후1년경기전망CSI',
        '업황실적BSI(전산업)', '업황실적BSI(제조업)', 'Tone_score'
    ]
)

# ============================
# 5. 스케일링
scaler_y = Scaler()
scaler_x = Scaler()

series_scaled = scaler_y.fit_transform(series)
covariates_scaled = scaler_x.fit_transform(covariates)

# ============================
# 6. train / test 분리
train_series = series_scaled.slice(pd.Timestamp("2018-01-02"), pd.Timestamp("2024-12-31"))
test_series = series_scaled.slice(pd.Timestamp("2025-01-01"), pd.Timestamp("2025-07-31"))

train_cov = covariates_scaled.slice(pd.Timestamp("2018-01-02"), pd.Timestamp("2024-12-31"))
test_cov = covariates_scaled.slice(pd.Timestamp("2025-01-01"), pd.Timestamp("2025-07-31"))

In [11]:
# 7. TFT 모델 정의
model = TFTModel(
    input_chunk_length=60,
    output_chunk_length=30,
    hidden_size=32,
    lstm_layers=1,
    dropout=0.1,
    batch_size=32,
    n_epochs=50,
    add_relative_index=True,
    random_state=42,
    pl_trainer_kwargs={
        "accelerator": "cpu",    
        "precision": "32-true"   
    }
)

In [12]:
# 8. 모델 학습
model.fit(
    series=train_series,
    past_covariates=train_cov,
    verbose=True
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/kook/myenv/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 28.8 K
5  | decoder_vsn                       | _VariableSelectionNetwork        | 896   
6  | sta

Training: |                                               | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


TFTModel(output_chunk_shift=0, hidden_size=32, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=True, loss_fn=None, likelihood=None, norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=60, output_chunk_length=30, batch_size=32, n_epochs=50, random_state=42, pl_trainer_kwargs={'accelerator': 'cpu', 'precision': '32-true'})

In [13]:
# 9. 테스트셋 예측
forecast = model.predict(
    n=len(test_series),
    past_covariates=covariates_scaled
)

# 스케일 역변환
forecast = scaler_y.inverse_transform(forecast)
test_series = scaler_y.inverse_transform(test_series)

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |                                             | 0/? [00:00<?, ?it/s]

In [ ]:
# 10. 성능 평가
print("RMSE:", rmse(test_series, forecast))
print("MAE:", mae(test_series, forecast))
print("R²:", r2_score(test_series, forecast))

In [ ]:
# 11. 시각화
plt.figure(figsize=(10,5))
test_series.plot(label="Actual (2025)")
forecast.plot(label="Predicted (2025)")
plt.title("KOFR Forecasting with TFT")
plt.legend()
plt.show()